# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / l_s + (r_c ** 2) / r_s, axis=1) / (l_s + r_s).ravel()
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # чтобы избежать варнингов сделаем под логарифмом всегда не 0
        return -np.sum(l_c * np.log(l_c / l_s + 1e-8) + r_c * np.log(r_c / r_s + 1e-8), axis=1) / (l_s + r_s).ravel() 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(1) + r_c.max(1)) / (l_s + r_s).ravel()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортирует по возрастанию значений выбранного признака x согласованно с классами y в выборке
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Находит количество уникальных классов в выборке, которую передали на вход
        class_number = np.unique(y).shape[0]
        
        # Отступим от концов массива sorted_y так, чтобы никакой класс в разделении не оказался листом
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # Находим такие значения i: splitted_sorted_y[i] != splitted_sorted_y[i + 1], 
        # то есть правые концы групп подряд идущих элементов из одного класса в splitted_sorted_y.
        # Сместим так, чтобы индексы найденных концов соотносились с индексами в sorted_y
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Если все классы в splitted_sorted_y одинаковые, разбиение не требуется
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Количество элементов в группах подряд идущих элементов из одного класса
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # Находит матрицу class_increments, строки - номера концов из r_border_ids, столбцы - классы.
        # В [i, j] ячейке записано число элементов класса j, которые перейдут в левую часть разбиения,
        # когда мы повысим порог так, чтобы взять в эту часть ещё одну i группу. 
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # В l_class_count[i, j] количество элементов j класса которые попали в левый класс 
        # при пороге x <= sorted_x[r_border_ids[i]] (но вообще говоря при некоторых соседних i этот порог может
        # оказаться одинаковым, что может оказаться проблемой, так как задание такого порога даст не тот результат,
        # который ожидали)
        l_class_count = np.cumsum(class_increments, axis=0) 
        # Аналогично для правого класса вычитанием из общего количества элементов j класса
        r_class_count = np.bincount(y) - l_class_count
        # Размеры левого класса при разбиениях
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        # Размеры правых классов
        r_sizes = sorted_y.shape[0] - l_sizes

        # Вычисляет меру неопределённости для возможных разбиений и выбирает разбиение с минимальной мерой
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Для полученного индекса оптимального разбиения возвращает меру неопределённости и значение порога
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __add_leaf_node_to_tree(self, y, node_id):
        distr = np.bincount(y, minlength=self.num_class)
        self.tree[node_id] = (self.LEAF_TYPE, distr.argmax(), distr / float(distr.sum()))
        
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if x.shape[0] <= 2 * self.min_samples_split + 1 or depth == self.max_depth:
            self.__add_leaf_node_to_tree(y, node_id)
            return
        
        if np.bincount(y).max() / float(y.shape[0]) > self.sufficient_share:
            self.__add_leaf_node_to_tree(y, node_id)
            return
        
        min_gs = float('+inf')
        optimal_feature_id = -1
        optimal_feature_treshold = 0
        
        # Здесь борьба с тем, что в функцию __find_threshold можно передавать только такие y, в которых 
        # есть все классы от 0 до какого-то class_number
        y_temp = np.where(y.reshape(-1, 1) == np.unique(y))[1]

        for feature in self.get_feature_ids(x.shape[1]):
            gs, treshold = self.__find_threshold(x[:, feature], y_temp)
            if gs < min_gs:
                min_gs = gs
                optimal_feature_id = feature
                optimal_feature_treshold = treshold
        
        if optimal_feature_id == -1:
            self.__add_leaf_node_to_tree(y, node_id)
            return
          
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, optimal_feature_id, optimal_feature_treshold)
        
        if y_l.size == 0 or y_r.size == 0:
            self.__add_leaf_node_to_tree(y, node_id)
            return
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, optimal_feature_id, optimal_feature_treshold)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=15)
clf = DecisionTreeClassifier(min_samples_split=2, max_depth=15, criterion='gini')

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.483999967575
0.859999895096


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.933108838447
0.932900972811
0.930115573293
0.930531304565
0.93335550659


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.925500956182
0.922258252266
0.924669493639
0.922175106011
0.922088720742


Но в данной выборке сильно преобладает класс 0, для более качественной оценки возьмём поровну представителей 0 и 1 класса

In [10]:
min_count = np.min(np.bincount(y))
x_equal_count = np.empty((0, x.shape[1]))
y_equal_count = np.empty((0), dtype='int64')
permutation = np.random.permutation(y.size)
x = x[permutation]
y = y[permutation]
for class_number in np.unique(y):
    ind = np.where(y == class_number)[0][:min_count]
    x_equal_count = np.append(x_equal_count, x[ind], axis=0)
    y_equal_count = np.append(y_equal_count, y[ind], axis=0)
permutation = np.random.permutation(y_equal_count.size)
x_equal_count = x_equal_count[permutation]
y_equal_count = y_equal_count[permutation]

In [11]:
for train, test in gkf.split(x_equal_count, y_equal_count):
    X_train, y_train = x_equal_count[train], y_equal_count[train]
    X_test, y_test = x_equal_count[test], y_equal_count[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.734071193539
0.739156446306
0.747532156745
0.75142087945
0.745062836625


In [12]:
for train, test in gkf.split(x_equal_count, y_equal_count):
    X_train, y_train = x_equal_count[train], y_equal_count[train]
    X_test, y_test = x_equal_count[test], y_equal_count[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.72629374813
0.727789410709
0.719413700269
0.70894406222
0.739676840215


В целом удалось добиться того, что реализация из sklearn работает дольше и показывает качество хуже. Хотя, возможно, если в классификатор sklearn передавать подобранные параметры, то можно улучшить скорость и качество.